## Create a Spark Session:
`.config("spark.ui.port", "4040")` creates a Spark UI under Port 4040

In [1]:
from pyspark.sql import SparkSession

spark = SparkSession.builder \
    .appName("NYC Taxi Insights") \
    .master("local[*]") \
    .config("spark.ui.port", "4040") \
    .getOrCreate()

### Reduce number of partitions for low-level PCs

In [2]:
# Standardwert ist 200
spark.conf.get("spark.sql.shuffle.partitions")
# -> '200'

'200'

In [3]:
# Anpassung
spark.conf.set("spark.sql.shuffle.partitions", "20")

### load data

In [4]:
df = spark.read.parquet("C:/Data/NYC_TLC_Trip_Records/yellow_tripdata_2025-01.parquet")
df.printSchema()
df.show(5)

root
 |-- VendorID: integer (nullable = true)
 |-- tpep_pickup_datetime: timestamp_ntz (nullable = true)
 |-- tpep_dropoff_datetime: timestamp_ntz (nullable = true)
 |-- passenger_count: long (nullable = true)
 |-- trip_distance: double (nullable = true)
 |-- RatecodeID: long (nullable = true)
 |-- store_and_fwd_flag: string (nullable = true)
 |-- PULocationID: integer (nullable = true)
 |-- DOLocationID: integer (nullable = true)
 |-- payment_type: long (nullable = true)
 |-- fare_amount: double (nullable = true)
 |-- extra: double (nullable = true)
 |-- mta_tax: double (nullable = true)
 |-- tip_amount: double (nullable = true)
 |-- tolls_amount: double (nullable = true)
 |-- improvement_surcharge: double (nullable = true)
 |-- total_amount: double (nullable = true)
 |-- congestion_surcharge: double (nullable = true)
 |-- Airport_fee: double (nullable = true)
 |-- cbd_congestion_fee: double (nullable = true)

+--------+--------------------+---------------------+---------------+------

In [5]:
import os

total_size = 0
directory = r"C:\Data\NYC_TLC_Trip_Records"

for filename in os.listdir(directory):
    filepath = os.path.join(directory, filename)
    if os.path.isfile(filepath):
        size_mb = os.path.getsize(filepath) / (1024 * 1024)  # in MB
        print(f"{filename:40} {size_mb:10.2f} MB")
        total_size += size_mb

print(round(total_size),"MB")

yellow_tripdata_2025-01.parquet               56.42 MB
yellow_tripdata_2025-02.parquet               57.55 MB
yellow_tripdata_2025-03.parquet               66.72 MB
181 MB


In [6]:
from pyspark.sql.functions import col, count, avg, max, min

# Anzahl der Fahrten
total_trips = df.count()

# Durchschnittliche Distanz und Dauer
avg_distance = df.agg(avg("trip_distance")).first()[0]
avg_fare = df.agg(avg("fare_amount")).first()[0]

print(f"🚖 Gesamtfahrten: {total_trips:,}")
print(f"📏 Durchschnittliche Distanz: {avg_distance:.2f} miles")
print(f"💵 Durchschnittlicher Fahrpreis: ${avg_fare:.2f}")

🚖 Gesamtfahrten: 3,475,226
📏 Durchschnittliche Distanz: 5.86 miles
💵 Durchschnittlicher Fahrpreis: $17.08


In [ ]:
from pyspark.sql.functions import hour, dayofweek

# Anzahl Fahrten pro Stunde
trips_per_hour = df.withColumn("hour", hour("tpep_pickup_datetime")) \
    .groupBy("hour").count().orderBy("hour")
trips_per_hour.show()

In [ ]:
import matplotlib.pyplot as plt

# Fahrten pro Stunde
pdf = trips_per_hour.toPandas()

plt.figure(figsize=(10,5))
plt.bar(pdf["hour"], pdf["count"], color="steelblue")
plt.xlabel("Stunde des Tages")
plt.ylabel("Anzahl Fahrten")
plt.title("NYC Yellow Taxi: Fahrten pro Stunde")
plt.show()

In [ ]:
pickup_counts = df.groupBy("PULocationID").count().orderBy(col("count").desc())
pickup_counts.show(10)